<a href="https://colab.research.google.com/github/Luke-zm/coursera_learning/blob/main/Dynamic_Economic_Dispatch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# For colab, install the necessary kits
!pip install pyomo
!apt-get install -y -qq glpk-utils
!apt install coinor-libipopt-dev
# !apt install coinor-libipopt-dev
# !pip install ipopt
!pip install ipopt
!wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
coinor-libipopt-dev is already the newest version (3.11.9-2.2build5).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.


In [2]:

# Import the solver tools
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

# Import the processing tool
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:

# Note: This is an econmic dispatch problem
# For all generating units, there will be an input output curve...
# This curve is approximated by:
# fuel_cost = alpha*Power_gen**2 + beta*power_gen + gamma
# This a quadratic function

# Below is the table of such an equation
generation_dict = {"generator": ['G1', 'G2', 'G3', 'G4', 'G5'],
                   "a": [3.00, 4.05, 4.05, 3.99, 3.88],
                   "b": [20.00, 18.07, 15.55, 19.21, 26.18],
                   "c": [100.00, 98.87, 104.26, 107.21, 95.31],
                   "pmin": [28, 90, 68, 76, 19],
                   "pmax": [206, 284, 189, 266, 53]}
generation_df = pd.DataFrame(generation_dict)
generation_df

,generator,a,b,c,pmin,pmax
0,G1,3.00,20.00,100.00,28,206
1,G2,4.05,18.07,98.87,90,284
2,G3,4.05,15.55,104.26,68,189
3,G4,3.99,19.21,107.21,76,266
4,G5,3.88,26.18,95.31,19,53


In [4]:
# Note for a Dynamic Economic dispatch, where power gen depends also on time t
dw_dict = {
    "t":['t1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9', 't10', 't11', 't12', 't13', 't14', 't15', 't16', 't17', 't18', 't19', 't20', 't21', 't22', 't23', 't24'],
    "D":[0.970176268, 0.373409762, 0.359247182, 0.717889635, 0.373826945, 0.885041091, 0.947937799, 0.302320872, 0.342469483, 0.836228617, 0.754609393, 0.443983539,
         0.236877761, 0.419213396, 0.966441323, 0.898253174, 0.570751561, 0.920452007, 0.449079919, 0.474674464, 0.613894557, 0.886955747, 0.715683494, 0.460268155],
    "W":[0.768629531, 0.206747050, 0.473333487, 0.809360223, 0.227406488, 0.570545816, 0.903537634, 0.980649211, 0.104204787, 0.294911145, 0.116396279, 0.738295254,
         0.490862683, 0.883447154, 0.370330075, 0.497099838, 0.474839097, 0.336632337, 0.935600839, 0.275455539, 0.198305128, 0.146477488, 0.650905851, 0.312793437]
}
dw_df = pd.DataFrame(dw_dict)
dw_df

,t,D,W
0,t1,0.970176,0.768630
1,t2,0.373410,0.206747
2,t3,0.359247,0.473333
3,t4,0.717890,0.809360
4,t5,0.373827,0.227406
5,t6,0.885041,0.570546
6,t7,0.947938,0.903538
7,t8,0.302321,0.980649
8,t9,0.342469,0.104205
9,t10,0.836229,0.294911


$$   
\\ min~ \sum_{g,t}a_g P_{g,t}^2 + b_gP_{g,t}+c_g    
\\     
\\subject~to:~   
\\    
\\P_g^{min}\leq P_{g,t}\leq P_g^{max}    
\\     
\\ \sum P_{g,t} \geq Load_t   
\\   
\\ P_{g,t+1} - P_{g,t} \geq RU_g    
\\     
\\ P_{g,t} - P{g,t+1} \geq RD_g    
$$  

In [5]:
print(type(generation_df['generator']))
print(type(list(generation_df['generator'])))

<class 'pandas.core.series.Series'>
<class 'list'>


In [6]:
# Prepare the model
# Initialize an AbstractModel()
model = pyo.AbstractModel()
# Initialize the indices
model.g = pyo.Set(initialize=list(generation_df['generator']))
# Initialize the time indices
# model.t = pyo.Set(initialize=list(dw_df['t']))
######
# It will be very annoying when dealing with time steps if your indices are str
# This is becasue you can't easily shift between the indices using +/-
# Define the time step as int indices
model.t = pyo.RangeSet(len(dw_df))

In [7]:

# The model need to create different upper and lower bounds for Pg
# For this purpose, a function is needed
# All functions pyomo need to have model object passed in
# since that is where things are being defined
def bound_Pg(model, g, t):
  pmin = generation_df.loc[generation_df['generator']==g,'pmin']
  pmax = generation_df.loc[generation_df['generator']==g,'pmax']
  return (float(pmin), float(pmax))
# Initialize the Variable Pg with dynamic bounds using the above function
model.Pg = pyo.Var(model.g, model.t, bounds=bound_Pg, within=pyo.NonNegativeReals)

In [8]:

# Define the loads
# This is a parameter and the demand is considered to be 300 MW
model.Load = pyo.Param(initialize=300, within=pyo.NonNegativeReals)

In [9]:
print(dw_df.loc[dw_df.index==1,'D'])

1    0.37341
Name: D, dtype: float64


In [10]:
type(float(dw_df.loc[dw_df.index==1,'D']))

float

In [11]:
# Define the constraints for economic dispatch

# Define a function for the economic dispatch
def power_balance(model, t, g=None):
  return sum(model.Pg[g, t] for g in model.g) >= model.Load*float(dw_df.loc[dw_df.index==1,'D'])
model.power_balance = pyo.Constraint(model.t, rule=power_balance)

In [12]:
#Define the ramp up constraint
def rule_RU(model,g,t):
  if t+1 <= 24:
    return model.Pg[g, t+1] - model.Pg[g,t] <= 0.1*float(generation_df.loc[generation_df['generator']==g,'pmax'])
  else:
    return pyo.Constraint.Skip
model.ramp_up = pyo.Constraint(model.g, model.t, rule=rule_RU)

In [13]:
#Define the ramp down constraint
def rule_RD(model,g,t):
  if t+1 <= 24:
    return model.Pg[g, t] - model.Pg[g,t+1] <= 0.1*float(generation_df.loc[generation_df['generator']==g,'pmax'])
  else:
    return pyo.Constraint.Skip
model.ramp_down = pyo.Constraint(model.g, model.t, rule=rule_RD)

In [14]:
# Define the objective function
def objective_func(model):
  return sum(float(generation_df.loc[generation_df['generator']=='G1','a']) * model.Pg[g, t] * model.Pg[g, t] +
             float(generation_df.loc[generation_df['generator']=='G1','b']) * model.Pg[g, t] +
             float(generation_df.loc[generation_df['generator']=='G1','c']) for g in model.g
             for t in model.t)

# Use the objective function
model.objective_func = pyo.Objective(rule=objective_func, sense=pyo.minimize)

In [15]:
result = SolverFactory('ipopt')
instance = model.create_instance()

In [16]:
# result = SolverFactory('ipopt')
# instance = model.create_instance()
results = result.solve(instance)

In [17]:
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 254
  Number of variables: 120
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: Ipopt 3.12.13\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.04394030570983887
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0


In [18]:
for t in instance.t:
  for g in instance.g:
    print(t, g, pyo.value(instance.Pg[g, t]))

1 G1 28.0
1 G2 90.0
1 G3 68.0
1 G4 76.0
1 G5 19.0
2 G1 28.0
2 G2 90.0
2 G3 68.0
2 G4 76.0
2 G5 19.0
3 G1 28.0
3 G2 90.0
3 G3 68.0
3 G4 76.0
3 G5 19.0
4 G1 28.0
4 G2 90.0
4 G3 68.0
4 G4 76.0
4 G5 19.0
5 G1 28.0
5 G2 90.0
5 G3 68.0
5 G4 76.0
5 G5 19.0
6 G1 28.0
6 G2 90.0
6 G3 68.0
6 G4 76.0
6 G5 19.0
7 G1 28.0
7 G2 90.0
7 G3 68.0
7 G4 76.0
7 G5 19.0
8 G1 28.0
8 G2 90.0
8 G3 68.0
8 G4 76.0
8 G5 19.0
9 G1 28.0
9 G2 90.0
9 G3 68.0
9 G4 76.0
9 G5 19.0
10 G1 28.0
10 G2 90.0
10 G3 68.0
10 G4 76.0
10 G5 19.0
11 G1 28.0
11 G2 90.0
11 G3 68.0
11 G4 76.0
11 G5 19.0
12 G1 28.0
12 G2 90.0
12 G3 68.0
12 G4 76.0
12 G5 19.0
13 G1 28.0
13 G2 90.0
13 G3 68.0
13 G4 76.0
13 G5 19.0
14 G1 28.0
14 G2 90.0
14 G3 68.0
14 G4 76.0
14 G5 19.0
15 G1 28.0
15 G2 90.0
15 G3 68.0
15 G4 76.0
15 G5 19.0
16 G1 28.0
16 G2 90.0
16 G3 68.0
16 G4 76.0
16 G5 19.0
17 G1 28.0
17 G2 90.0
17 G3 68.0
17 G4 76.0
17 G5 19.0
18 G1 28.0
18 G2 90.0
18 G3 68.0
18 G4 76.0
18 G5 19.0
19 G1 28.0
19 G2 90.0
19 G3 68.0
19 G4 76.0
19 G5 19.0
